### Импорт библиотек.

In [99]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [100]:
np.random.seed(0)

### Загрузка данных, извлечение комментариев и целевой переменной в отдельные датафреймы.

In [101]:
df = pd.read_csv("data/data.csv")

In [102]:
df_comments = df["comment_text"]
df_target = (df["target"] > 0.7).astype(int)

In [103]:
df_comments.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
90902 non-null  object
dtypes: object(1)
memory usage: 710.3+ KB


In [104]:
df_target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 90902 entries, 0 to 90901
Series name: target
Non-Null Count  Dtype
--------------  -----
90902 non-null  int64
dtypes: int64(1)
memory usage: 710.3 KB


### Задание 1. Разделение данных на обучаемый и тестовый наборы данных.

In [105]:
X_train, X_test, Y_train, Y_test = train_test_split(df_comments, df_target, test_size=0.3, random_state=42)

### Задание 2. Преобразование данных в числовой формат.

In [106]:
vectorizer = CountVectorizer()

X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

### Задание 3. Импортирт из библиотеки ```sklearn``` логистической регрессии ```LogisticRegression```, обучение модели и вычисление метрики ```accuracy```.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train_vect, Y_train)

Y_pred = model.predict(X_test_vect)

accuracy = accuracy_score(Y_test, Y_pred)
print(f'Точность модели на тестовой выборке составляет {accuracy:.4f}')

Точность модели на тестовой выборке составляет 0.93


### Задание 4. Определение функции, вычисляющей токсичность комментария.

In [108]:
def predict(model, vectorizer, comment):
  comment_vect = vectorizer.transform([comment])
  return model.predict_proba(comment_vect)[0, 1]

comment = 'You\'re an idiot'
score = predict(model, vectorizer, comment)
print(f'Вероятность токсичности комментария «{comment}» составляет {score:.4f}')

Вероятность токсичности комментария «You're an idiot» составляет 0.9996


### Задание 5. Предсказание токсичности комментариев «Apples are stupid» и «I love apples».

In [109]:
comments = [ 'Apples are stupid', 'I love apples' ]

for comment in comments:
  score = predict(model, vectorizer, comment)
  print(f'Вероятность токсичности комментария «{comment}» составляет {score:.4f}')

Вероятность токсичности комментария «Apples are stupid» составляет 0.9991
Вероятность токсичности комментария «I love apples» составляет 0.0585


### Задание 6. Вывод десять наиболее токсичных слов и их коэффициентов.

In [110]:
coefs = model.coef_[0]
feature_names = vectorizer.get_feature_names_out()

word_coefs = list(zip(feature_names, coefs))
word_coefs_sorted = sorted(word_coefs, key=lambda x: x[1], reverse=True)

for word, coef in word_coefs_sorted[:10]:
  print(f"{word}: {coef:.4f}")

stupid: 9.2285
idiot: 8.7215
idiots: 8.4467
stupidity: 7.5361
idiotic: 6.8372
crap: 6.5751
dumb: 6.4499
pathetic: 6.4187
hypocrite: 6.3901
moron: 6.3562


### Задание 7. Анализ слов на токсичность.

На наш взгляд, все эти слова могут быть, вполне, исключены из списка токсичных, поскольку во-первых, без конкретного контекста невозможно однозначно сказать является ли данное слово токсичным или нет. Во-вторых, какие-то из этих слов могут являться частью устойчивых выражений, смысл которых не является токсичным.

Приведем несколько примеров. Так, слова ```stupid```, ```stupidity```, ```idiot```, ```idiots```, ```idiotic``` - это не толлько ```тупой```, ```тупость```, ```идиот``` или ```идиотизм```, а еще и ```глупость```, ```глупый```, ```бестолковый```, ```бестолковость```, ```нелепость``` и т. д. А слово ```crap```, которое часто используют как ```дерьмо``` - это также ```чепуха```, ```чушь```.

Таким образом если посмотреть переводы всех этих слов, то они помимо негативной оценки имеют общеупотребительное нейтральное значение. И еще раз повторим, что даже негативные токсичные значения нельзя назвать таковыми, не знаю к кому они относятся и в какой ситуации.

### Задание 8. Исследование модели на предвзятость.

In [111]:
comments = [ 'I have a christian friend', 'I have a muslim friend', 'I have a white friend', 'I have a black friend' ]

for comment in comments:
  score = predict(model, vectorizer, comment)
  print(f'Вероятность токсичности комментария «{comment}» составляет {score:.4f}')

Вероятность токсичности комментария «I have a christian friend» составляет 0.1879
Вероятность токсичности комментария «I have a muslim friend» составляет 0.5125
Вероятность токсичности комментария «I have a white friend» составляет 0.4074
Вероятность токсичности комментария «I have a black friend» составляет 0.5924


### Задание 9. Определение типа предвзятости модели.

Рассматриваемый случай, когда комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, относится к ```демографическо1 предвзятости```.

Однако, по моему мнению, в случае, когда соблюдены все условия для непредвзятости модели, но при этом она, тем не менее, выдает предвзятый результат, говорит о том, что общество по отношению к данной группе людей настроено отрицательно или, как минимум, настороженно. Как правило это негативное отношение свзяанно с объективными причинами "поведения" данной группы людей. То есть, нельзя игнорировать предвзятость модели в угоду толерантности и решать проблему ее игнорируя или подгоняя под нужный результат.


### Задание 10. Улучшение этичности алгоритма.

- Количественное выравнинание токсичных и нетоксичных комментариев для разных групп людей.
- Использование более сложных моделей, которые учитывают контекст высказываний.
- Переодическое тестирование модели на предвзятость в отношении каких-либо групп людей.